# Data preparation

* From **business understanding**, we know the task to be solved.  
* Then we do **data understanding** to look into data.
* Now we are going to do some necessary or useful data transformation to reach the aim.

## Outline
0. Summary of data understanding
1. Missing and invalid data
2. Feature extraction
3. Making different statistical units
4. Data transformation

## Data and tasks
* Titanic2 (*titanic_train.csv*) - data preparation for an analysis of ticket fares
* Home Credit (*application_train.csv*) - segmentation of clients by family situation

In [ ]:
# setup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_theme(style="ticks", color_codes=True)

## Part I. Titanic and ticket fares
### Summary of data understanding
Just few facts from the exploration -- for the aim of this practice.

Let's consider these columns only: *pclass*, *sex*, *age*, *ticket*, *fare*, *cabin*, *embarked*

In [ ]:
# data reading
df1 = pd.read_csv('titanic_train.csv')
df1 = df1[['passenger_id', 'ticket', 'pclass', 'fare', 'sex', 'age', 'cabin', 'embarked']]
df1

In [ ]:
# share of missing data (NaN, NULL) by columns
print(1 - df1.count()/len(df1))

* *ticket*, *pclass* and *sex* are complete
* *fare* and *embarked* have negligible counts of missing data
* *age* and *cabin* have significant counts of missing data

In [ ]:
# invalid values in data?
# frequency tables of categorical columns
print(df1['pclass'].value_counts())
print(df1['sex'].value_counts())
print(df1['embarked'].value_counts())
# the most often values in string columns
print(df1['ticket'].value_counts().sort_values(ascending=False)[:5])
print(df1['cabin'].value_counts().sort_values(ascending=False)[:5])

> String columns (*ticket*, *cabin*) have expected frequencies -- no value has too high frequency.  
> Categorical columns seem to have valid values.

Let's look into numeric columns (*age*, *fare*).

In [ ]:
# distribution of values in numeric columns
sns.displot(df1['fare'])
print('Fare: minimum=', df1['fare'].min(), '; maximum=', df1['fare'].max(), '; median=', df1['fare'].median())

In [ ]:
# zero fare is rather unexpected; how many passenger have zero fare?
print('Passengers with zero fare: ', (df1['fare']==0).sum())
print('The most often fares: ')
print(df1['fare'].value_counts().sort_values(ascending=False).iloc[0:5])

> Fare values seem to be valid with exception of zero and missing values.

In [ ]:
sns.displot(df1['age'])
print('Age: minimum=', df1['age'].min(), '; maximum=', df1['age'].max(), '; median=', df1['age'].median())
print('The most often ages:')
print(df1['age'].value_counts().sort_values(ascending=False).iloc[0:5])

> Age values seem to be fully valid with exception of missing values.

### Dealing with missing and invalid data

Now we use exploration outcomes for the data cleaning.

**TASK 1.**  
Consider how to treat missing or invalid data of fare, embarkment, age and cabin. Then prepare a script for data cleaning.

**Answers:**

* missing fare -- could be either omitted (one case only) or estimated from other attributes
* zero fare -- few cases only, could be kept as valid (possibly special passengers) or omitted (possibly errors)
* missing embarkment -- could be either be omitted (one case only) or estimated from other attributes
* missing age -- should not be omitted (too many cases, we have to deal with it other way)
* missing cabin -- should not be omitted (missing value is informative)

In [ ]:
# cleaning - example of omitting records with missing values (keeping record with non-missing and valid values)
# we will not run it yet
# df1 = df1[df1['fare'].notna() & (df1['fare']>0) & (df1['embarked'].notna())]

### Feature extraction

Multiple persons travelled on one ticket, so they can have the same fare which was paid only once. It's a reason to make new statistical units &ndash; tickets. But is data for the same ticket consistent? Let's check the integrity of data for the tickets.

**TASK 2.**  
Explore whether all passengers with the same ticket have the same fare, pclass, embarkment and cabin.

In [ ]:
# same fare for the same ticket?
print(df1.groupby('ticket').agg({'fare': 'nunique'}).value_counts())
# Which ticket is for a passenger with missing fare? Are there more passengers for this ticket?
ticket_na_fare = df1[df1['fare'].isna()]['ticket'].values.tolist()
df1[df1['ticket'].isin(ticket_na_fare)]

In [ ]:
# same pclass for the same ticket?
print(df1.groupby('ticket').agg({'pclass': 'nunique'}).value_counts())

In [ ]:
# same embarkments for the same ticket?
print(df1.groupby('ticket').agg({'embarked': 'nunique'}).value_counts())
# For which ticket were there more embarkments?
tmp_tickets = df1.groupby('ticket').agg({'embarked': 'nunique'})
ticket_mult_emb = tmp_tickets[tmp_tickets['embarked'] > 1].index.tolist()
df1[df1['ticket'].isin(ticket_mult_emb)]

In [ ]:
# same cabin for the same ticket?
print(df1.groupby('ticket').agg({'cabin': 'nunique'}).value_counts())

> For each ticket, there is the same fare (possibly missing or zero) and same class.  
> For each ticket except two cases, there is one embarkment place. One ticket has two places and one ticket none (missing).  
> There can be various numbers of cabin for a ticket (and possibly none, too).

Now we make a table of tickets by few steps:

1. Base table -- unique rows of *ticket*, *pclass*, *fare* (we now there is integrity).
2. Aggregated features grouped by *ticket* -- e. g. count of passengers; join aggregated table to the base table.
3. Artificial aggregation as a solution of multiple embarkment -- we take the highest value of *embarked* to unify embarkment places for tickets.

**TASK 3.**  
Make a table with tickets as rows and features (some of them aggregated). Choose useful features for future analysis by yourself.

**Chosen features:**
* count of passengers
* ratio of male passengers
* age of the youngest and of the oldest passenger
* average age of passengers
* count af passengers with known age
* sex od the oldest passenger
* count of (distinct) cabins

In [ ]:
# User function
def rate_males(s):
    return np.mean(np.where(s=='male', 1, 0))

### Base table
df2_base = df1[['ticket', 'pclass', 'fare']].drop_duplicates()
df2_base = df2_base.set_index('ticket') # setting 'ticket' column as key

### Multiple embarkment solution
df2_emb = df1.groupby('ticket').agg({'embarked': 'max'})
# print('Ticket with multiple embarkment has been unified:')
# print(df2_emb.loc['113798'])
# no need to set index - groupby + agg sets index by default

### Some chosen features
df2_feat = df1.groupby('ticket').agg({'ticket': 'count', 'sex': [rate_males],
                                      'age': ['min', 'max', np.mean, 'count'], 'cabin': 'nunique'})
# column names update
df2_feat.columns = ['pass_cnt', 'rate_males', 'age_min', 'age_max', 'age_mean', 'age_valid_cnt', 'cabin_cnt']

# sex of the oldest person for the ticket
df2_feat_sex_oldest = df1.sort_values(by=['ticket', 'age'], ascending=[True, False]) \
    .drop_duplicates('ticket')[['ticket', 'sex']]
df2_feat_sex_oldest = df2_feat_sex_oldest.set_index('ticket') # setting 'ticket' column as key
df2_feat_sex_oldest.columns = ['sex_oldest']

### Joining tables together
df2 = df2_base.join(df2_emb) # join is by default LEFT and index<->index
df2 = df2.join(df2_feat)
df2 = df2.join(df2_feat_sex_oldest)

df2

### Data transformation

* The distribution of fare is very skew. Let's transform it by log to get it better balanced.
* The fare is given as a total. But it's better to get an average fare per one passenger.

**TASK 4.**
Add new columns to the table as stated above.

In [ ]:
# we use log10 for better interpretation, but simple log is ok, too
# be careful at zero fare - log is invalid! (we can use log(x+1) instead)
df2['fare_log'] = np.log10(df2['fare']+1)
df2['fare_per_pass'] = df2['fare'] / df2['pass_cnt']
df2

**TASK 5.**
1. Make new columns as meaningful categories "binned" from count of passengers, mean age, count of distinct cabins.
2. Make flags "child" and "baby": flag is True when the youngest passenger for a ticket was under 15, resp. under 3 years.
3. Find the most often combinations of men and women travelling on one ticket (e. g. "single man", "man+woman", "two men", "other" etc.) and make a new column with category description.

In [ ]:
# First, find proper breaks for binning; then make binning; then check frequencies

### pass_cnt
print(df2['pass_cnt'].value_counts())
# categories will be 1, 2, 3, 4+
df2['pass_cnt_cat'] = pd.cut(df2['pass_cnt'], [0, 1, 2, 3, 1000], labels=['1', '2', '3', '4+'])
print(df2['pass_cnt_cat'].value_counts())

### age_mean
sns.displot(df2['age_mean'])
# categories will be up to 15, 15-25, 25-40, 40+
df2['age_mean_cat'] = pd.cut(df2['age_mean'], [0, 15, 20, 25, 30, 40, 1000],
                             labels=['15-', '15-20', '20-25', '25-30', '30-40', '40+'])
print(df2['age_mean_cat'].value_counts().sort_index())

### cabin_cnt (same approach as pass_cnt)
print(df2['cabin_cnt'].value_counts())
# categories will be none, 1, 2+
df2['cabin_cnt_cat'] = pd.cut(df2['cabin_cnt'], [0, 1, 2, 1000], right=False, labels=['none', '1', '2+'])
print(df2['cabin_cnt_cat'].value_counts())

In [ ]:
# Flags child and baby
df2['flag_child'] = (df2['age_min'] < 15)
df2['flag_baby'] = (df2['age_min'] < 3)

In [ ]:
# Analyze frequencies of various combinations
pd.pivot_table(df2, values='fare', index=['pass_cnt_cat'], columns=['rate_males'], aggfunc=np.size)

# then make a new column
# very simple example - can be extended by nested np.where conditions
df2['group'] = np.where((df2['rate_males']==1) & (df2['pass_cnt']==1), 'Single man', 'Other')

df2

## Part II. Home credit
This dataset contains Home Credit clients who got a loan. Each client (=row in the dataset) has plenty of data in columns. We are interested in the segmentation of client portfolio. Segmentation is a division the basic dataset into some well-defined segment, like "young single men", "old widow women living alone" etc.

The relevant columns are *days_birth*, *code_gender*, *cnt_children*, *cnt_fam_members*, *name_family_status*.

**TASK: look into data and try to find some big segments based on some features from the set of relevant columns. You may need to do some binning before.**

In [ ]:
df_hc = pd.read_csv('application_train.csv')
df_hc.columns = df_hc.columns.str.lower()
df_hc = df_hc[['days_birth', 'code_gender', 'cnt_children', 'cnt_fam_members', 'name_family_status']]
df_hc